In [1]:
!pip install roboflow
!pip install ultralytics

# CityScapes
from roboflow import Roboflow
rf = Roboflow(api_key="eVeIuFmVXuAoqDnlp9kf")
project = rf.workspace("hienlongairesearch").project("city-scapes")
version = project.version(3)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.7 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sigstore 3.6.1 requires rich~=13.0, but you have rich 14.0.0 which is incompatible.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, 


Extracting Dataset Version Zip to City-scapes-3 in yolov8:: 100%|██████████| 6539/6539 [00:01<00:00, 4039.76it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
import torch
import torch.nn as nn
from ultralytics import YOLO
from ultralytics.nn.modules import Conv

path = '/kaggle/working/City-scapes-3/data.yaml'

# DyCAF-Net w/o Equilibrium Neck

In [3]:
class ModifiedYOLO_WithoutNeck(YOLO):
    """Uses original YOLOv8 neck (NO DyCAFNeck replacement)"""
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)  # Keep original neck
        # Remove any neck replacement logic

model = ModifiedYOLO_WithoutNeck('yolov8s.pt')

model.train(
    data=path,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)

100%|██████████| 21.5M/21.5M [00:00<00:00, 155MB/s] 


Ultralytics 8.3.110 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/City-scapes-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 16.1MB/s]
E0000 00:00:1744926662.843459      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744926662.911943      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.0MB/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:08<00:00,  2.52it/s]


                   all        653       5731      0.327      0.255      0.248      0.154
                ignore        490       1557      0.272     0.0636      0.052      0.016
            pedestrian        534       3251      0.558       0.63       0.64      0.439
        person -other-         72        104       0.18     0.0769     0.0334     0.0119
          person group         79        150          0          0     0.0067    0.00237
                 rider        233        402      0.521      0.633      0.617      0.416
        sitting person         84        267       0.43      0.124      0.138     0.0409


invalid value encountered in less
invalid value encountered in less


Speed: 0.1ms preprocess, 6.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c9cc9977b10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

# DyCAF-Net w/o Dual Attention

In [4]:
class SimpleBlock(nn.Module):
    """Block without Channel & Spatial Attention"""
    def __init__(self, channels, depth=3):
        super().__init__()
        self.convs = nn.ModuleList([Conv(channels, channels, k=3, act="SiLU") for _ in range(depth)])
    
    def forward(self, x):
        identity = x
        for conv in self.convs:
            x = conv(x)
        return identity + x

class DyCAFNeckWithoutDualAttention(nn.Module):
    """Neck using SimpleBlock instead of DyCAFBlock"""
    def __init__(self, backbone_channels=[256, 512, 1024], depth=2):
        super().__init__()
        # Keep the same structure as DyCAFNeck but use SimpleBlock
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.downsample = nn.MaxPool2d(2, stride=2)
        
        self.c5_conv = Conv(backbone_channels[2], backbone_channels[2], 1)
        self.c4_conv = Conv(backbone_channels[1], backbone_channels[1], 1)
        self.c3_conv = Conv(backbone_channels[0], backbone_channels[0], 1)
        
        self.top_down_blocks = nn.ModuleList([
            SimpleBlock(backbone_channels[2], depth),
            SimpleBlock(backbone_channels[1], depth),
            SimpleBlock(backbone_channels[0], depth)
        ])
        
        self.bottom_up_blocks = nn.ModuleList([
            SimpleBlock(backbone_channels[0], depth),
            SimpleBlock(backbone_channels[1], depth),
            SimpleBlock(backbone_channels[2], depth)
        ])
    
    # Forward method remains the same as DyCAFNeck

class ModifiedYOLO_WithoutDualAttention(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        backbone_channels = [
            self.model.model[6].cv2.conv.out_channels,
            self.model.model[8].cv2.conv.out_channels,
            self.model.model[9].cv1.conv.out_channels
        ]
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = DyCAFNeckWithoutDualAttention(backbone_channels=backbone_channels)
                break
# For DyCAF-Net w/o Dual Attention
model = ModifiedYOLO_WithoutDualAttention('yolov8s.pt')

model.train(
    data=path,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)

Ultralytics 8.3.110 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/City-scapes-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning /kaggle/working/City-scapes-3/train/labels.cache... 2612 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2612/2612 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1051.6±916.5 MB/s, size: 184.8 KB)


val: Scanning /kaggle/working/City-scapes-3/valid/labels.cache... 653 images, 0 backgrounds, 0 corrupt: 100%|██████████| 653/653 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:08<00:00,  2.57it/s]


                   all        653       5731      0.327      0.255      0.248      0.154
                ignore        490       1557      0.272     0.0636      0.052      0.016
            pedestrian        534       3251      0.558       0.63       0.64      0.439
        person -other-         72        104       0.18     0.0769     0.0334     0.0119
          person group         79        150          0          0     0.0067    0.00237
                 rider        233        402      0.521      0.633      0.617      0.416
        sitting person         84        267       0.43      0.124      0.138     0.0409


invalid value encountered in less
invalid value encountered in less


Speed: 0.1ms preprocess, 6.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c9cc9e08090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

# DyCAF-Net w/o Class-Aware Adaptation

In [5]:
class CAFAHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv = Conv(in_channels, in_channels, k=3, act="SiLU")
        self.attention = nn.Sequential(
            nn.Conv2d(in_channels, num_classes, kernel_size=1),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        return x * self.attention(self.conv(x))

# 2. Define SimpleHead
class SimpleHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv = Conv(in_channels, in_channels, k=3, act="SiLU")
    
    def forward(self, x):
        return self.conv(x)

# 3. Define DyCAFNeck (if not already defined elsewhere)
class DyCAFNeck(nn.Module):
    def __init__(self, backbone_channels=[256, 512, 1024], depth=2):
        super().__init__()
        # ... (keep your original DyCAFNeck implementation here)

# 4. Define ModifiedYOLO parent class
class ModifiedYOLO(YOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_neck()
    
    def replace_neck(self):
        backbone_channels = [...]  # Your backbone channel logic
        for i, layer in enumerate(self.model.model):
            if isinstance(layer, nn.Sequential):  
                self.model.model[i] = DyCAFNeck(backbone_channels=backbone_channels)
                break

# 5. Define child class
class ModifiedYOLO_WithoutClassAware(ModifiedYOLO):
    def __init__(self, model_path='yolov8s.yaml'):
        super().__init__(model_path)
        self.replace_head()
    
    def replace_head(self):
        head = self.model.model[-1]  # Access detection head
        for name, module in head.named_children():
            if isinstance(module, CAFAHead):
                in_channels = module.conv.conv.in_channels
                num_classes = module.attention[0].out_channels
                setattr(head, name, SimpleHead(in_channels, num_classes))

# Initialize and train
model = ModifiedYOLO_WithoutClassAware('yolov8s.yaml')  # Use YAML config

model.train(
    data=path,
    epochs=50,
    batch=16,
    imgsz=640,
    lr0=0.01,
    lrf=0.1,
    optimizer="SGD",
    augment=True,
    mosaic=True,
    weight_decay=0.0005,
    dropout=0.5
)

New https://pypi.org/project/ultralytics/8.3.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.110 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/kaggle/working/City-scapes-3/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.5, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fal

train: Scanning /kaggle/working/City-scapes-3/train/labels.cache... 2612 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2612/2612 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 802.1±740.5 MB/s, size: 184.8 KB)


val: Scanning /kaggle/working/City-scapes-3/valid/labels.cache... 653 images, 0 backgrounds, 0 corrupt: 100%|██████████| 653/653 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:08<00:00,  2.57it/s]


                   all        653       5731      0.638       0.19      0.206      0.128
                ignore        490       1557       0.31     0.0199     0.0236     0.0083
            pedestrian        534       3251      0.621      0.545      0.578      0.377
        person -other-         72        104          1          0     0.0191    0.00458
          person group         79        150          1          0    0.00386    0.00116
                 rider        233        402      0.589       0.53      0.549       0.36
        sitting person         84        267       0.31     0.0449     0.0628     0.0185


invalid value encountered in less
invalid value encountered in less


Speed: 0.1ms preprocess, 6.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/train3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c9cd572d350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     